In [2]:
import os
import sys
import torch
import torchvision

from torch import nn
from pathlib import Path
from torchinfo import summary
from torchvision import transforms, datasets

from modules import engine, utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def create_effnetb2_model(num_classes: int=3):
    """Creates an EfficientNetB2 feature extractor model and transforms
    
    Args:
        num_classes: number of classes in the classifier head, defaults to 3
        
    Returns:
        model: EffNetB2 feature extractor model
        transforms: EffNetB2 image transforms
    """
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)

    for param in model.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes)
    )

    return model, transforms

In [4]:
effnetb2_food101, effnetb2_transforms = create_effnetb2_model(num_classes=101)
print(summary(
    effnetb2_food101, 
    input_size=(1, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
))

food101_train_transforms = transforms.Compose([
    transforms.TrivialAugmentWide(),
    effnetb2_transforms,
])

venv_dir = Path(sys.prefix)
project_root = venv_dir.parent
data_path = project_root/"data"

train_data = datasets.Food101(
    root=data_path,
    split="train",
    transform=food101_train_transforms,
    download=True
)

test_data = datasets.Food101(
    root=data_path,
    split="test",
    transform=effnetb2_transforms,
    download=True
)

food101_class_names = train_data.classes


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 101]             --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

In [5]:
BATCH_SIZE = 32
NUM_WORKERS = 2 if os.cpu_count() <= 4 else 4

train_dataloader_food101 = torch.utils.data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True
)

test_dataloader_food101 = torch.utils.data.DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False
)

The original Food101 paper achieved a 56.4% accuracy on the test data. We will use label smoothing, a state-of-the-art regularization technique 

In [19]:
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(params=effnetb2_food101.parameters(), lr=1e-3)

effnetb2_food101_results = engine.train(
    model=effnetb2_food101,
    train_dataloader=train_dataloader_food101,
    test_dataloader=test_dataloader_food101,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=5,
    device=device
)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.9892 | train_acc: 0.4059 | test_loss: 2.1747 | test_acc: 0.6114
Epoch: 2 | train_loss: 2.6553 | train_acc: 0.4807 | test_loss: 2.1006 | test_acc: 0.6314
Epoch: 3 | train_loss: 2.6113 | train_acc: 0.4906 | test_loss: 2.0862 | test_acc: 0.6376
Epoch: 4 | train_loss: 2.6005 | train_acc: 0.4964 | test_loss: 2.0542 | test_acc: 0.6435
Epoch: 5 | train_loss: 2.5923 | train_acc: 0.4982 | test_loss: 2.0541 | test_acc: 0.6428


In [23]:
utils.save_model_with_savetensors(
    model=effnetb2_food101,
    model_name="effnetb2_food101.safetensors"
)

utils.save_model(
    model=effnetb2_food101,
    model_name="effnetb2_food101.pth"
)

Saving model to: /home/glauco/Desktop/projects/learningPyTorch/trained_models/effnetb2_food101.pth


30

In [6]:
food101_class_names

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheese_plate',
 'cheesecake',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster